# Menambah Library

Memasukan library yang dibutuhkan dalam projek

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import winsound

from tensorflow.python.client import device_lib
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Cek Spek

Mengetahui apakah hanya dapat memakai CPU atau dapat menggunakan CPU dan GPU (Nvidia Cuda)

In [ ]:
device_lib.list_local_devices()

# AUGMENTASI GAMBAR

Generator untuk membuat gambar menjadi lebih variatif dan menentukan besar data yang digunakan untuk validasi

In [ ]:
datagen_train = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    fill_mode = 'nearest',
    validation_split = 0.2
)

datagen_val = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

# DATASET

Ubah data menjadi dataset training & validation

In [ ]:
dataset = 'C:/Users/Hanif/Downloads/Compressed/27 Cat Breeds'

jumlah_batch = 32
ukuran_gambar = (120, 120)

train = datagen_train.flow_from_directory(
    dataset,
    target_size=ukuran_gambar,
    batch_size=jumlah_batch,
    class_mode='categorical',
    subset='training'
)

val = datagen_val.flow_from_directory(
    dataset,
    target_size=ukuran_gambar,
    batch_size=jumlah_batch,
    class_mode='categorical',
    subset='validation'
)

# MODEL CNN

Membuat model dengan algoritma cnn untuk klasifikasi gambar multi kelas

In [ ]:
jumlah_kelas = val.num_classes

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(ukuran_gambar[0], ukuran_gambar[1], 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.25)(x)

predictions = Dense(jumlah_kelas, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.summary()

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# LATIH

Model yang sudah dibuat dilatih memakai dataset training Dan validation

In [ ]:
akurasi_minimal = 60e-2
jumlah_langkah = int(train.samples/train.batch_size)

class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('val_accuracy') >= akurasi_minimal:
            self.model.stop_training = True
            
latih = model.fit(
    train,
    steps_per_epoch=jumlah_langkah,
    epochs=5000,
    validation_data=val,
    callbacks=[my_callbacks()]
)

winsound.Beep(1500,1000)

Grafik riwayat pelatihan untuk melihat seberapa baik model dilatih

In [ ]:
akurasi = latih.history['val_accuracy']
loss = latih.history['val_loss']

plt.plot(akurasi, color='blue', label='Akurasi')
plt.plot(loss, color='red', label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Nilai')
plt.title('Riwayat Pelatihan Model')
plt.legend()
plt.show()

Cek akurasi akhir model

In [ ]:
hasil = model.evaluate(val)

accuracy = hasil[1]
print(f'\nAkurasi Model: {accuracy * 100:.2f} %')

# Evaluasi

Mencoba model dengan data nyata diluar dataset

In [ ]:
import os

daftar_kelas = list(val.class_indices.keys())

daftar_file = os.listdir('Examples')

for file in daftar_file:
    gambar = os.path.join('Examples/', file)

    img = tf.keras.utils.load_img(gambar, target_size=ukuran_gambar)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    jenis = daftar_kelas[tf.argmax(score)]

    display = tf.keras.utils.load_img(gambar, target_size=None)
    plt.imshow(display)
    plt.xlabel(f'Prediksi : {jenis}\n Asli : {file[:-4]}', fontsize=10)
    plt.show()